In [2]:
#!pip3 install cassandra-driver

In [3]:
# Importing packages

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import csv
import uuid

In [4]:
# From console it is assigned 
ASTRA_DB_ID = '19cf8906-f987-4b74-a9a9-1ca42c56e000'
ASTRA_DB_REGION = 'us-east1'
ASTRA_DB_APPLICATION_TOKEN = 'AstraCS:oooCMShTTrtnyocJlEpAnSEr:3233df889ee8dedfcc3c4a252d3e676ff09821828f40d0c247df5628f7e7e666'

# Defined at the time of database creation
ASTRA_DB_KEYSPACE = 'insurance_keyspace'
ASTRA_DB_COLLECTION= 'insurance_collection'

# From generated token file
ASTRA_CLIENT_ID = 'oooCMShTTrtnyocJlEpAnSEr' 
ASTRA_CLIENT_SECRET = '87ZyISoBoXZSkgtZfHIq1P-C9y+PzZpwHMQLfxXJ6n0SbyS9tJRx2+ju4h.1QCu,lNYzPZYhHm6K6M.c-oH7UMt27Z.yuvxUPU4czu48qlzQZvt5XZ4ePwpy6SSHBPd6'

In [5]:
# Connecting to Astra Database

cloud_config = {'secure_connect_bundle': r"secure-connect-insurance.zip"}

cloud_config

{'secure_connect_bundle': 'secure-connect-insurance.zip'}

In [6]:
# Creating auth_provider

auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)

auth_provider

In [7]:
# Cluster is a collection of nodes (servers) that work together to store and manage data

Cluster = Cluster(cloud= cloud_config, auth_provider= auth_provider)

Cluster

In [8]:
session = Cluster.connect()

In [9]:
drop_table_cql = f"DROP TABLE IF EXISTS {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION}"

In [10]:
# Creating table for data loading in keyspace-> collection
create_table_cql = f"CREATE TABLE IF NOT EXISTS {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION} (\
                    customer_id UUID PRIMARY KEY, \
                    age INT, \
                    sex VARCHAR, \
                    bmi FLOAT, \
                    children INT, \
                    smoker VARCHAR, \
                    region VARCHAR, \
                    expenses FLOAT);" 

In [11]:
# Executing quaries in cql (Cassandra query language)
session.execute(drop_table_cql)
session.execute(create_table_cql)

In [12]:
# Defining csv_file name
csv_file = "insurance.csv"

In [13]:
# Preparing the INSERT INTO quary with placeholders

insert_query = f"INSERT INTO {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION} " \
                "(customer_id, age, sex, bmi, children, smoker, region, expenses)" \
                "VALUES(?, ?, ?, ?, ?, ?, ?, ?);"

In [14]:
# Appending data to keyspace(database container) row by row
with open(csv_file, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        customer_id = uuid.uuid4()
        age = int(row["age"])
        sex = row['sex']
        bmi = float(row['bmi'])
        children = int(row['children'])
        smoker = row["smoker"]
        region = row['region']
        expenses = float(row['expenses'])

        # Execute the INSERT query with parameterized values
        prepared = session.prepare(insert_query)
        session.execute(prepared, (customer_id, age, sex, bmi, children, smoker, region, expenses))
    print("Success!")

Success!


In [15]:
rows = session.execute(f"SELECT * FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION}")

for row in rows[:20]:
    # customer_id, age, bmi, children, expenses, region, sex, smoker
    print(row[0], row[1], round(row[2],2), row[3], round(row[4],2), row[5], row[6], row[7])

5a9754e8-1c52-40f3-8f7d-21e316ac61ee 55 21.5 1 10791.96 southwest male no
2450426f-7116-41ac-ac23-582c05a54adc 18 17.3 2 12829.46 northeast male yes
869cd050-0852-4cf9-843b-6c7f1da77976 48 22.8 0 8269.04 southwest female no
319a13db-21c0-446b-b7db-32bd9d115b0f 21 17.4 1 2585.27 southwest female no
a9571397-6c1e-4e2d-855a-4db5ee3118d9 64 23.8 0 26926.51 southeast male yes
ba3bcc3a-7db2-4521-bd01-577ffbd6877b 54 28.9 2 12096.65 northeast female no
3a62b5cf-d365-43d8-9e87-6472536e270d 63 27.0 0 28950.47 northwest female yes
f151c92c-abb4-4a55-aebc-24cfe2ca5ef8 23 28.3 0 18033.97 northwest female yes
99a1167c-3a3c-4edc-b677-07cdbf51f481 64 39.2 1 14418.28 southeast male no
02907b47-85d2-43b0-83ef-6ea2902bc800 59 31.8 2 12928.79 southeast male no
232074b8-002e-40ed-82c8-b633a0ccb767 25 25.8 1 3309.79 northeast male no
b93423d2-9813-4ce7-94c0-0c9cf556993e 30 35.3 0 36837.47 southwest male yes
f343be04-9ce0-4418-afdb-7b9effac2a0b 19 33.1 0 23082.96 southwest male no
18cdc647-122a-4ce6-8f11-ee